# Chapter 23. Databases and SQL

In [1]:
from __future__ import division
import math, random, re
from collections import defaultdict

The data that you will be working with will often live in [databases](https://en.wikipedia.org/wiki/Database), systems designed for efficiently storing and querying data.  
The bulk of these are [relational databases](https://en.wikipedia.org/wiki/Relational_database) such as [Oracle](https://en.wikipedia.org/wiki/Oracle_Database), [MySQL](https://en.wikipedia.org/wiki/MySQL), and [SQL Server](https://en.wikipedia.org/wiki/Microsoft_SQL_Server), which store data in [tables](https://en.wikipedia.org/wiki/Table_%28database%29) and are typically queried using [Structured Query Language (SQL)](https://en.wikipedia.org/wiki/SQL), a [declarative language](https://en.wikipedia.org/wiki/SQL) for manipulating data.  
SQL is an essential part of a data scientist's toolkit.  
In this chapter, we'll create NotQuiteABase, a Python implementation of something that's not quite a database.  
We'll also cover the basics of SQL while demonstrating how those principles work in NotQuiteABase.  
Hopefully, solving problems in NotQuiteABase will give you a good sense of how you might solve the same problems using SQL.

## CREATE TABLE and INSERT

A relational database is a collection of tables and the relationships among those tables.  
A table is simply a collection of rows, not unlike the matrices we've been working with.  
However, a table also has a fixed [database schema](https://en.wikipedia.org/wiki/Database_schema) consisting of column names and column types.  
For example, imagine a `users` data set containing (for each user) her `user_id, name,` and `num_friends`:

In SQL, we might create this table with:

Notice that we specified that the `user_id` and `num_friends` must be integers (and that `user_id` can't be NULL, which indicates a missing value and is sort of like Python's `None`) and that the name should be a string of length 200 or less.  
NotQuiteABase won't take types into account, but we'll behave as if it did.  
Also SQL doesn't usually care about case (you don't have to capitalize SELECT or GROUP BY) or indentation, so the style you use here will probably be different than styles you encounter elsewhere.

You can insert the rows with INSERT statements:

Notice also that SQL statements need to end with semicolons, and that SQL requires single quotes for its strings.  
In NotQuiteABase, you'll create a `Table` simply by specifying the names of its columns.  
To insert a row, you'll use the table's `insert()` method, which takes a `list` of row values that need to be in the same order as the table's column names.  
Behind the scenes, we'll store each row as a `dict` from column names to values.  
A real database would never use such a space-wasting representation, but doing so will make NotQuiteABase much easier to work with:

In [2]:
# For reasons I haven't figured out yet, the notebook gives me attitude when I separate the function definition cells
# from the cell that the class is defined in, so here is the entire class definition.
# I will cover the function definitions and their descriptions/explanations below as the chapter progresses.

class Table:


    def __init__(self, columns):
        self.columns = columns
        self.rows = []


    def __repr__(self):
        """ pretty representation of the table: first columns then rows """
        return str(self.columns) + "\n" + "\n".join(map(str, self.rows))


    def __getitem__(self, user):
        """ return row for specified user: users[user] """
        return self.rows[user]


    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("Wrong Number of Elements")
        row_dict = dict(zip(self.columns, row_values))
        self.rows.append(row_dict)


    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.iteritems():
                    row[column] = new_value


    def delete(self, predicate=lambda row: True):
        """ delete all rows matching predicate or all rows if no predicate is given """
        self.rows = [row for row in self.rows if not(predicat(row))]


    def select(self, keep_columns=None, additional_columns=None):
        # if no columns are specified, return all columns
        if keep_columns is None:
            keep_columns = self.columns
        if additional_columns is None:
            additional_columns = {}

        # create new table for results
        result_table = Table(keep_columns + additional_columns.keys())

        for row in self.rows:
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.iteritems():
                new_row.append(calculation(row))
            result_table.insert(new_row)

        return result_table


    def where(self, predicate=lambda row: True):
        """ return only the rows that satisfy the supplied predicate """
        where_table = Table(self.columns)
        where_table.rows = filter(predicate, self.rows)
        return where_table


    def limit(self, num_rows=None):
        """ return only the first num_rows rows """
        limit_table = Table(self.columns)
        limit_table.rows = (self.rows[:num_rows]
                            if num_rows is not None
                            else self.rows)
        return limit_table


    def group_by(self, group_by_columns, aggregates, having=None):

        grouped_rows = defaultdict(list)

        # populate groups
        for row in self.rows:
            key = tuple(row[column] for column in group_by_columns)
            grouped_rows[key].append(row)
            
        # result table consists of group_by columns and aggregates
        result_table = Table(group_by_columns + aggregates.keys())

        for key, rows in grouped_rows.iteritems():
            if having is None or having(rows):
                new_row = list(key)
                for aggregate_name, aggregate_fn in aggregates.iteritems():
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)

        return result_table


    def order_by(self, order):
        # make a copy of the table
        new_table = self.select()
        new_table.rows.sort(key=order)
        return new_table


    def join(self, other_table, left_join=False):
        # join on columns in both tables
        join_on_columns = [c for c in self.columns
                           if c in other_table.columns]
        # join on columns only in right table
        additional_columns = [c for c in other_table.columns
                              if c not in join_on_columns]
        # join all columns from left table + additional_columns from right table
        join_table = Table(self.columns + additional_columns)

        for row in self.rows:

            def is_join(other_row):
                return all(other_row[c] == row[c] for c in join_on_columns)

            other_rows = other_table.where(is_join).rows

            # each other row that matches this one produces a result row
            for other_row in other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [other_row[c] for c in additional_columns])
            # if no rows match and it's a left join, output with Nones
            if left_join and not other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [None for c in additional_columns])

        return join_table

For example, we could set up:

In [3]:
    users = Table(["user_id", "name", "num_friends"])
    users.insert([0, "Hero", 0])
    users.insert([1, "Dunn", 2])
    users.insert([2, "Sue", 3])
    users.insert([3, "Chi", 3])
    users.insert([4, "Thor", 3])
    users.insert([5, "Clive", 2])
    users.insert([6, "Hicks", 3])
    users.insert([7, "Devin", 2])
    users.insert([8, "Kate", 2])
    users.insert([9, "Klein", 3])
    users.insert([10, "Jen", 1])

In [4]:
print users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


## UPDATE

Sometimes you need to update the data that's already in the database.  
For instance, if Dunn acquires another friend, you might need to do this:

The key features are:
- What table to update 
- Which rows to update
- Which fields to update
- What their new values should be  

We'll add a similar `update` method to NotQuiteABase.  
Its first argument will be a `dict` whose keys are the columns to update and whose values are the new values for those fields.  
Its second argument is a [predicate](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29) that returns `True` for rows that should be updated, `False` otherwise:

In [5]:
def update(self, updates, predicate):
    for row in self.rows:
        if predicate(row):
            for column, new_value in updates.iteritems():
                row[column] = new_value

Now when Dunn makes a new friend and we want to update his information, we can do this:

In [6]:
# set num_friends = 3 in rows where user_id == 1
users.update({'num_friends' : 3 }, lambda row: row['user_id'] == 1)
users[1]

{'name': 'Dunn', 'num_friends': 3, 'user_id': 1}

## DELETE

There are two ways to delete rows from a table in SQL.  
The dangerous way deletes every row from a table:

The less dangerous way adds a WHERE clause and only deletes rows that match a certain condition:

Let's add this functionality to our `Table`:

In [7]:
def delete(self, predicate=lambda row: True):
    """ deletes all rows matching the predicate or all rows if no predicate is specified """
    self.rows = [row for row in self.rows if not(predicate(row))]

For example: 

## SELECT

Typically you don't inspect SQL tables directly.  
Instead you query them with a SELECT statement:

You can also use SELECT statements to calculate fields:

We'll give our `Table` class a `select()` method that returns a new `Table`.  
This method accepts two optional arguments:
- `keep_columns` specifies the name of the columns that you want to keep in the result. If you don't supply it, the result contains all of the columns.
- `additional_columns` is a `dict` whose keys are new column names and whose values are functions specifying how to compute the values of the new columns.  

If you were to supply neither of those arguments, you would simply get back a copy of the table:

In [8]:
def select(self, keep_columns=None, additional_columns=None):
    # if no columns are specified 
    if keep_columns is None:
        # return all columns
        keep_columns = self.columns 
    
    if additional_columns is None:
        additional_columns = {}
    
    # new table for results 
    result_table = Table(keep_columns + additional_columns.keys())
    
    for row in self.rows:
        new_row = [row[column] for column in keep_columns]
        for column_name, calculation in additional_columns.iteritems():
            new_row.append(calculation(row))
        result_table.insert(new_row)
        
    return result_table

Our `select()` method returns a new `Table`, while the typical SQL `SELECT` just produces some sort of transient result set (unless you explicitly insert the results into a table).

We're also going to need `where()` and `limit()` methods:

In [9]:
def where(self, predicate=lambda row: True):
    """ return only those rows that satisfy the supplied predicate """
    where_table = Table(self.columns)
    where_table.rows = filter(predicate, self.rows)
    return where_table

def limit(self, num_rows):
    """ return only the first num_rows rows """
    limit_table = Table(self.columns)
    limit_table.rows = self.rows[:num_rows]
    return limit_table

Now we can construct NotQuiteABase equivalents for the preceding SQL statements:

In [10]:
print "SELECT * FROM users;"
print users.select()
print

print "SELECT * FROM users LIMIT 2;"
print users.limit(2)
print

print "SELECT user_id FROM users;"
print users.select(keep_columns=["user_id"])
print

print "SELECT user_id FROM users WHERE name == 'Dunn';"
print users.where(lambda row: row["name"] == "Dunn").select(keep_columns=["user_id"])
print

print "SELECT LENGTH(name) AS name_length FROM users;"
def name_length(row): return len(row["name"])
print users.select(keep_columns=[], additional_columns = { "name_length" : name_length })
print

SELECT * FROM users;
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 3}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

SELECT * FROM users LIMIT 2;
['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 3}

SELECT user_id FROM users;
['user_id']
{'user_id': 0}
{'user_id': 1}
{'user_id': 2}
{'user_id': 3}
{'user_id': 4}
{'user_id': 5}
{'user_id': 6}
{'user_id': 7}
{'user_id': 8}
{'user_id': 9}
{'user_id': 10}

SELECT user_id FROM users WHERE n

# GROUP BY

Anoither common SQL operation is GROUP BY, which groups together rows with identical values in specified columns and produces aggregate values like MIN, MAX, COUNT, and SUM.  
This should remind you of the `group_by` function from "Manipulating Data" in Chapter 10.
For example, you might want to find the number of users and the smallest `user_id` for each possible name length:

Every field we SELECT needs to be either in the GROUP BY clause (which `name_length` is) or an aggregate computation (which `min_user_id` and `num_users` are).
SQL supports a HAVING clause that behaves similarly to a WHERE clause except that its filter is applied to the aggregates, whereas a WHERE would filter out rows before aggregation even took place.  
You might want to know the average number of friends for users whose names start with specific letters but only see the results for letters whose corresponding average is greater than 1.  
I know, some of these examples are contrived, but they help with the learning process.

Note that functions for working with strings vary across SQL implementations; some databases might use SUBSTRING instead of SUBSTR, and so on.  
You can also calculate overall aggregates.  
In that case, you leave off the GROUP BY:

Now, to add this functionality to NotQuiteABase `Tables`, we'll add a `group_by()` method.  
This method takes the names of the columns that you want to group by, a dictionary of the aggregation functions that you want to run over each group, and an optional predicate `having` that operates on multiple rows.  
Then `group_by()` does the following steps:
1. Creates a `defaultdict` to map `tuples` (of the group-by-values) to rows (containing the group-by-values). Remember that you can't use lists as `dict` keys, you must use tuples instead.
2. Iterates over the rows of the table, populating the `defaultdict`.
3. Creates a new table with the correct output columns.
4. Iterates over the `defaultdict` and populates the output table, applying the `having` filter (if any).  

Note that an actual database does this in a (hopefully) much more efficient manner.

In [11]:
def group_by(self, group_by_columns, aggregates, having=None):
    grouped_rows = defaultdict(list)
    # populate the groups
    for row in self.rows:
        key = tuple(row[column] for column in group_by_columns)
        grouped_rows[key].append(row)
    # result table consists of group_by columns and aggregates
    result_table = Table(group_by_columns + aggregates.keys())
    
    for key, rows in grouped_rows.iteritems():
        if having is None or having(rows):
            new_row - list(key)
            for aggregate_name, aggregate_fn in aggregates.iteritems():
                new_row.append(aggregate_fn(rows))
            result_table.insert(new_row)
            
    return result_table

Now then, let's see how we would do the equivalent of the preceding SQL statements:

In [12]:
print "SELECT LENGTH(name) as name_length,"
print " MIN(user_id) AS min_user_id,"
print " COUNT(*) AS num_users"
print "FROM users"
print "GROUP BY LENGTH(name);"

def min_user_id(rows): return min(row["user_id"] for row in rows)
stats_by_length = users \
                  .select(additional_columns={"name_length" : name_length}) \
                  .group_by(group_by_columns=["name_length"], 
                            aggregates={ "min_user_id" : min_user_id,
                                         "num_users" : len })
print stats_by_length
print

print "SELECT SUBSTR(name, 1, 1) AS first_letter,"
print " AVG(num_friends) AS avg_num_friends"
print "FROM users"
print "GROUP BY SUBSTR(name, 1, 1)"
print "HAVING AVG(num_friends) > 1;"

def first_letter_of_name(row):
    return row["name"][0] if row["name"] else ""

def average_num_friends(rows):
    return sum(row["num_friends"] for row in rows) / len(rows)

def enough_friends(rows):
    return average_num_friends(rows) > 1
    # This will exclude user 10 but not user 0
    
avg_friends_by_letter = users \
                        .select(additional_columns={'first_letter' : first_letter_of_name }) \
                        .group_by(group_by_columns=['first_letter'],
                                  aggregates={ "avg_num_friends" : average_num_friends },
                                  having=enough_friends)
print avg_friends_by_letter
print 

print "SELECT SUM(user_id) as user_id_sum"
print "FROM users"
print "WHERE user_id > 1;"

def sum_user_ids(rows): return sum(row["user_id"] for row in rows)

user_id_sum = users \
              .where(lambda row: row["user_id"] > 1) \
              .group_by(group_by_columns=[],
                        aggregates={ "user_id_sum" : sum_user_ids })
print user_id_sum

SELECT LENGTH(name) as name_length,
 MIN(user_id) AS min_user_id,
 COUNT(*) AS num_users
FROM users
GROUP BY LENGTH(name);
['name_length', 'num_users', 'min_user_id']
{'num_users': 4, 'min_user_id': 5, 'name_length': 5}
{'num_users': 3, 'min_user_id': 2, 'name_length': 3}
{'num_users': 4, 'min_user_id': 0, 'name_length': 4}

SELECT SUBSTR(name, 1, 1) AS first_letter,
 AVG(num_friends) AS avg_num_friends
FROM users
GROUP BY SUBSTR(name, 1, 1)
HAVING AVG(num_friends) > 1;
['first_letter', 'avg_num_friends']
{'avg_num_friends': 3.0, 'first_letter': 'T'}
{'avg_num_friends': 2.5, 'first_letter': 'C'}
{'avg_num_friends': 1.5, 'first_letter': 'H'}
{'avg_num_friends': 2.5, 'first_letter': 'K'}
{'avg_num_friends': 2.5, 'first_letter': 'D'}
{'avg_num_friends': 3.0, 'first_letter': 'S'}

SELECT SUM(user_id) as user_id_sum
FROM users
WHERE user_id > 1;
['user_id_sum']
{'user_id_sum': 54}


## ORDER BY

Frequently, you'll want to sort your results.  
For example, you might want to know the (alphabetically) first two names of your users:

We can implement this by giving our `Table` an `order_by()` method that takes an `order` function:

In [13]:
def order_by(self, order):
    """ makes a copy of the table and then sorts it """
    new_table = self.select()
    new_table.rows.sort(key=order)
    return new_table

which we can use as follows:

In [14]:
print "SELECT * FROM users"
print "ORDER BY name"
print "LIMIT 2;"

friendliest_letters = avg_friends_by_letter \
                      .order_by(lambda row: -row["avg_num_friends"]) \
                      .limit(4)
print friendliest_letters

SELECT * FROM users
ORDER BY name
LIMIT 2;
['first_letter', 'avg_num_friends']
{'avg_num_friends': 3.0, 'first_letter': 'T'}
{'avg_num_friends': 3.0, 'first_letter': 'S'}
{'avg_num_friends': 2.5, 'first_letter': 'C'}
{'avg_num_friends': 2.5, 'first_letter': 'K'}


The SQL `ORDER BY` lets you specify ascending (ASC) or descending (DESC) for each sort field.  
Here we have to bake that into our order function by using (or omitting) the minus operator.

## JOIN

Relational database tables are often [normalized](https://en.wikipedia.org/wiki/Database_normalization), meaning that they're organized to minimize redundancy.  
For example, when we work with our users' interests in Python we can just give each user a `list` containing her interests.  
SQL tables can't usually contain lists, so the typical solution is to create a second table, like `user_interests`, containing the one-to-many relationship between `user_id`s and `user_interest`s.  
In SQL you might do this:

whereas in NotQuiteABase you would instead create a table like this:

In [15]:
user_interests = Table(["user_id", "interest"])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

In [16]:
user_interests

['user_id', 'interest']
{'user_id': 0, 'interest': 'SQL'}
{'user_id': 0, 'interest': 'NoSQL'}
{'user_id': 2, 'interest': 'SQL'}
{'user_id': 2, 'interest': 'MySQL'}

In our example, there's still plenty of redundancy -- the interest "SQL" is stored in two different places.  
In a real database you might store `user_id` and `interest_id` in the `user_interests` table and then create a third table `interests` that maps `interest_id` to `interest` so that you could store the interest names only once.  
However, that would make our examples more complicated than they need to be.

When our data lives across different tables, how do we analyze it?  
Well, we can JOIN the tables together.  
A JOIN combines rows in the left table with corresponding rows in the right table, where the meaning of "corresponding" is based on how we specify the join.  
For example, to find the users interested in SQL you would query:

The JOIN says that, for each row in users, we should look at the `user_id` and associate that row with every row in `user_interests` containing the same `user_id`.  
Notice that we had to specify which tables to JOIN and also which columns to join ON.  
This is an INNER JOIN, which returns the combinations of rows (and only the combinations of rows) that match according to the specified join criteria.  
There is also a LEFT JOIN, which -- in addition to the combinations of matching rows -- returns a row for each left-table row with no matching rows (in which case, the fields that would have come from the right table are all NULL).  
Using a LEFT JOIN, we can count the number of interests that each user has:

The LEFT JOIN ensures that users with no interests will still have rows in the joined data set (with NULL values for the fields coming from `user_interests`), and COUNT only counts values that are non-NULL.

The NotQuiteABase `join()` implementation will be more restrictive -- it simply joins two tables on whatever columns they have in common.  
Even so, the implementation is a bit complicated:

In [17]:
def join(self, other_table, left_join=False):
    # join on columns in both tables
    join_on_columns = [c for c in self.columns
                       if c in other_table.columns]
    # join on columns only in right table
    additional_columns = [c for c in other_table.columns
                          if c not in join_on_columns]
    # join all columns from left table + additional_columns from right table
    join_table = Table(self.columns + additional_columns)
    
    for row in self.rows:
        
        def is_join(other_row):
            return all(other_row[c] == row[c] for c in join_on_columns)
        
        other_rows = other_table.where(is_join).rows
        
        # each other row that matches this one produces a result row
        for other_row in other_rows:
            join_table.insert([row[c] for c in self.columns] +
                              [other_row[c] for c in additional_columns])
        # if no rows match and it's a left join, output with Nones
        if left_join and not other_rows:
            join_table.insert([row[c] for c in self.columns] +
                              [None for c in additional_columns])
            
    return join_table

Now we can find users interested in SQL with:

In [18]:
sql_users = users \
            .join(user_interests) \
            .where(lambda row: row["interest"] == "SQL") \
            .select(keep_columns=["name"])

sql_users

['name']
{'name': 'Hero'}
{'name': 'Sue'}

And we can find the interest counts with:

In [19]:
def count_interests(rows):
    """ counts how many rows have non-None interests """
    return len([row for row in rows if row["interest"] is not None])

user_interest_counts = users \
                       .join(user_interests, left_join=True) \
                       .group_by(group_by_columns=["user_id"],
                                 aggregates={"num_interests" : count_interests })
        
user_interest_counts

['user_id', 'num_interests']
{'user_id': 0, 'num_interests': 2}
{'user_id': 1, 'num_interests': 0}
{'user_id': 2, 'num_interests': 2}
{'user_id': 8, 'num_interests': 0}
{'user_id': 3, 'num_interests': 0}
{'user_id': 9, 'num_interests': 0}
{'user_id': 4, 'num_interests': 0}
{'user_id': 10, 'num_interests': 0}
{'user_id': 5, 'num_interests': 0}
{'user_id': 6, 'num_interests': 0}
{'user_id': 7, 'num_interests': 0}

In SQL, there is also a RIGHT JOIN, which keeps rows from the right table that have no matches, as well as a FULL OUTER JOIN, which keeps rows from both tables that have no matches.  
We won't implement either of those.

## Subqueries

In SQL, you can SELECT from (and JOIN) the results of queries as if they were tables.  
If you wanted to find the smallest `user_id` of anyone interested in SQL, you could use a subquery.  
Granted, you could do the same operation using a JOIN, but then we wouldn't be demonstrating subqueries.

Given the way we've designed NotQuiteABase, we get this for free, and our query results are actual tables:

In [20]:
likes_sql_user_ids = user_interests \
                     .where(lambda row: row["interest"] == "SQL") \
                     .select(keep_columns=['user_id'])
        
likes_sql_user_ids.group_by(group_by_columns=[],
                            aggregates={ "min_user_id" : min_user_id })

['min_user_id']
{'min_user_id': 0}

## Indexes

To find rows containing a specific value (say, where `name` is "Hero"), NotQuiteABase has to inspect every row in the table.  
If the table has a lot of rows, this can take a very long time.  
Similarly, our `join` algorithm is extremely inefficient.  
For each row in the left table, it inspects every row in the right table to see if it's a match.  
With two large tables, this could take approximately forever.  

Also, sometimes you want to apply constraints (search parameters) to some of your columns.  
For example, in your `users` table you probably don't want to allow two different users to have the same `user_id`.  
Indexes (indices?) solve those problems.  
If the `user_interests` table had an index on `user_id`, a smart `join` algorithm could find matches directly rather than scanning the entire table.  
If the `users` table had a 'unique' index on `user_id` (like [UUID or GUID](https://en.wikipedia.org/wiki/Universally_unique_identifier)), you would get an error if you tried to insert a duplicate.  

Each table in a database can have one or more indexes, which allow you to quickly look up rows by key columns, efficiently join tables together, and enforce unique constraints on columns or combinations of columns.  
Designing and using indexes well is somewhat of a black art which varies depending on the specific database, but if you end up doing a lot of database work, it's well worth learning about.

## Query Optimization

Recall the query we used to find all users who are interested in SQL:

In NotQuiteABase there are (at least) two different ways to write this query.  
You could filter the `user_interests` table before performing the join:

In [21]:
user_interests \
    .where(lambda row: row["interest"] == "SQL") \
    .join(users) \
    .select(["name"])

['name']
{'name': 'Hero'}
{'name': 'Sue'}

You could also filter the results of the join:

In [22]:
user_interests \
    .join(users) \
    .where(lambda row: row["interest"] == "SQL") \
    .select(["name"])

['name']
{'name': 'Hero'}
{'name': 'Sue'}

The results are the same either way, but filtering *before* the join (option 1) is almost certainly more efficient because `join` has many fewer rows to operate on than with option 2.  
In SQL, you generally wouldn't worry about this.  
You "declare" the results that you want and leave it up to the query engine to execute the commands (and use the indexes) as efficiently as possible.

## NoSQL

A recent trend in databases is toward nonrelational [NoSQL](https://en.wikipedia.org/wiki/NoSQL) databases, which don't represent data in tables.  
For example, MongoDB is a popular schema-less database whose elements are arbitrarily complex JSON documents rather than rows.  
There are:
- columnar databases that store data in columns instead of rows, which are useful for data that has many columns but queries only require a few of them.
- key-value stores that are optimized for retrieving single (complex) values by their keys (like a Python `dict`).
- databases for storing and traversing graphs.
- databases that are optimized to run across multiple datacenters.
- databases that are designed to run in memory.
- databases designed specifically for handling time-series data.
- and many more.  

Tomorrow's flavor of the day might not even exist now, so just know that NoSQL (and now [NewSQL](https://en.wikipedia.org/wiki/NewSQL) too) is a thing.  
Now you know, good luck keeping up to date.